### La base de données CSV au format PostgreSQL

In [12]:
import psycopg2

def ouvrir_connection(nom_bdd, utilisateur, mot_passe, host='localhost', port=5433):
    try:
        conn = psycopg2.connect(dbname=nom_bdd, user=utilisateur, password=mot_passe, host=host, port=5433)
    except psycopg2.Error as e:
        print("Erreur lors de la connection à la base de données")
        print(e)
        return None
    conn.set_session(autocommit=True)
    return conn


def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

    
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

    
def inserer_donnees(conn, sql_insertion_table, donnees):
    try:
        cursor = conn.cursor()
        
        for d in donnees:
            cursor.execute(sql_insertion_table, d)
            
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

    
def lire_donnees(conn, sql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data

def lire_csv_Dict(nom_fichier):
    data = []
    with open(nom_fichier, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
        
    return data


def renvoyer_annee_titre_film(title):
    new_title=title.strip()
    year=None
    try:
        if new_title[-6]=='(' and new_title[-1] == ')':
            year=int(new_title[-5:-1])
            new_title=new_title[:-6].strip()
    except:
        pass
    return  new_title,year

def update_movies(conn,movies,sql_add_annee,sql_update_title_annee):
    try:
        cursor = conn.cursor()
        #ajouter la colonne annee dans la table movies
        cursor.execute(sql_add_annee)
        
        for d in movies:
            titre_annee_formater = renvoyer_annee_titre_film(d[1])
            #print(titre_annee_formater[0],titre_annee_formater[1],d[0])
            cursor.execute(sql_update_title_annee,(titre_annee_formater[0],titre_annee_formater[1],d[0]))
           
        print("la colonne annees a bien été ajouté et le titre est formaté")
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    
    cursor.close()
        
    

In [5]:

from pathlib import Path
import csv

# Répertoire dataset (small)
repertoire_data = Path('data')

# Chemin vers les différents fichiers de données
chemin_movies = repertoire_data / 'movies.csv'
chemin_links = repertoire_data / 'links.csv'
chemin_ratings = repertoire_data / 'ratings.csv'
chemin_tags = repertoire_data / 'tags.csv'


#print(chemin_movies)




### Creation de BDD Floupics et ouverture de connection

In [13]:
import os
nom_bdd = "Floupics"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_psw')

conn = ouvrir_connection(nom_bdd, utilisateur, mot_passe)



### Importation des données et création des tables appropriées  

In [18]:
sql_suppression_table_movies = """ DROP TABLE IF EXISTS movies;"""

sql_creation_table='''create table if not exists movies(
                                       movieId int primary key,
                                       title text,
                                       genres text)'''

sql_insertion_table = '''insert into movies (movieId, title, genres) values (%(movieId)s, %(title)s,%(genres)s)''' 

sql_lecture_requete = '''select * from movies limit 50'''

sql_read_all = '''select * from movies'''

sql_add_annee = '''ALTER TABLE movies ADD COLUMN IF NOT EXISTS annee INT'''

sql_update_title_annee = '''UPDATE movies SET title= %s, annee= %s where movieId= %s'''



In [22]:
supprimer_table(conn, sql_suppression_table_movies)
creer_table(conn, sql_creation_table)

La table a été supprimée avec succès
La table a été crée avec succès


In [23]:
donnees = lire_csv_Dict(chemin_movies)
inserer_donnees(conn, sql_insertion_table, donnees)


Les données ont été insérées avec succès


In [4]:
#lire_donnees(conn, sql_lecture_requete)

In [24]:
#Ajouter la colonne annee et formater le titre dans la table movies:
donnee = lire_donnees(conn,sql_read_all)
update_movies(conn,donnee,sql_add_annee,sql_update_title_annee)


Les données ont été lues avec succès
la colonne annees a bien été ajouté et le titre est formaté


In [25]:
lire_donnees(conn, sql_lecture_requete)

Les données ont été lues avec succès


[(2, 'Jumanji', 'Adventure|Children|Fantasy', 1995),
 (3, 'Grumpier Old Men', 'Comedy|Romance', 1995),
 (4, 'Waiting to Exhale', 'Comedy|Drama|Romance', 1995),
 (5, 'Father of the Bride Part II', 'Comedy', 1995),
 (6, 'Heat', 'Action|Crime|Thriller', 1995),
 (7, 'Sabrina', 'Comedy|Romance', 1995),
 (8, 'Tom and Huck', 'Adventure|Children', 1995),
 (9, 'Sudden Death', 'Action', 1995),
 (10, 'GoldenEye', 'Action|Adventure|Thriller', 1995),
 (11, 'American President, The', 'Comedy|Drama|Romance', 1995),
 (12, 'Dracula: Dead and Loving It', 'Comedy|Horror', 1995),
 (13, 'Balto', 'Adventure|Animation|Children', 1995),
 (14, 'Nixon', 'Drama', 1995),
 (15, 'Cutthroat Island', 'Action|Adventure|Romance', 1995),
 (16, 'Casino', 'Crime|Drama', 1995),
 (17, 'Sense and Sensibility', 'Drama|Romance', 1995),
 (18, 'Four Rooms', 'Comedy', 1995),
 (19, 'Ace Ventura: When Nature Calls', 'Comedy', 1995),
 (20, 'Money Train', 'Action|Comedy|Crime|Drama|Thriller', 1995),
 (21, 'Get Shorty', 'Comedy|Crime|